In [1]:
import pandas as pd
import numpy as np
import pickle
import json
import import_ipynb
import scipy
from scipy.optimize import minimize, fmin_powell, fmin, differential_evolution
import warnings
from my_libs.calc_features import *
from sklearn.preprocessing import StandardScaler
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 100
from scipy.optimize import minimize

importing Jupyter notebook from C:\Users\kseniya.yuldasheva\Desktop\gitPNTZtoyuk_1\gitPNTZ\PNTZ\my_libs\calc_features.ipynb


In [161]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
path = "DATA/Optimizer/input.xlsx"

In [3]:
targets = ['Предел текучести',
           'Врем. сопротивление']

dir_names = ['DATA/MODELS_RF/YS RF valid', 
             'DATA/MODELS_RF/H RF valid']

models_bonds = [
    'Текучесть середина',
    'Прочность середина'
]

In [4]:
ls_columns_output = [
    '№ партии',
    '№ плавки',
    'марка стали',
    'диаметр',
    'толщина стенки',
    'Гр. прочн.',
    '1 зона по ВТР закалка',
    '2 зона по ВТР закалка',
    '3 зона по ВТР закалка',
    'шаг балок закалочная печь, сек',
    'Скорость прохождения трубы через спрейер, м/с', 
    't˚ C трубы после спреера',
    '1 зона ВТР и уставка отпуск', 
    '2 зона ВТР и уставка отпуск', 
    '3 зона ВТР и уставка отпуск',
    '4 зона ВТР и уставка отпуск',
    '5 зона ВТР и уставка отпуск',
    'шаг балок отпускная печь, сек',
    'C',
    'Mn',
    'Si',
    'P',
    'S',
    'Cr',
    'Ni',
    'Cu',
    'Al',
    'V',
    'Ti',
    'Nb',
    'Mo',
    'N',
    'B',
    'C-coef',
    'Параметр закалка',
    'Параметр отпуск',
    'Параметр отпуск новый V',
    'Величина зерна',
    'Тип предела текучести (1186)',
    'Дата термообработки',
    'ICD',
    'Примечание',
#     'длина трубы'
    ]

In [5]:
table_for_optimize = pd.read_excel(path, skiprows=1)

## Подготовка полученных от пользователя данных

#### Что будет вводить пользователь?  

Обязательно:
* группа прочности
* марка стали
* диаметр
* толщина стенки
* нижняя и верхняя границы допуска для предела текучести
* нижняя и верхняя границы допуска для предела прочности

Возможно:   
Режим термообработки, который нужно улучшить (он будет начальным приближением для алгоритма оптимизации):

* 1 зона по ВТР закалка
* 2 зона по ВТР закалка
* 3 зона по ВТР закалка
* Скорость прохождения трубы через спрейер, м/с
* 1 зона ВТР и уставка отпуск
* 2 зона ВТР и уставка отпуск
* 3 зона ВТР и уставка отпуск
* 4 зона ВТР и уставка отпуск
* 5 зона ВТР и уставка отпуск
* шаг балок закалочная печь, сек
* шаг балок отпускная печь, сек


Если режим термообработки не вводится, то будет подтягиваться исторический режим с минимальным шагом, в случае если шаг уже 24, режим оптимизироваться не будет (как в случае подтянутого исторического, так и в случае ручного ввода)

Сформируем списки обязательных и необязательных входных данных

In [6]:
ls_opt_need = [
    'Гр. прочн.',
    'марка стали',
    'толщина стенки',
    'диаметр'
]

bounds = [
    'Предел текучести нижняя граница',
    'Предел текучести верхняя граница',
    'Предел прочности нижняя граница',
    'Предел прочности верхняя граница'
]

ls_additional = [
    '1 зона по ВТР закалка',
    '2 зона по ВТР закалка',
    '3 зона по ВТР закалка',
    'Скорость прохождения трубы через спрейер, м/с',
    '1 зона ВТР и уставка отпуск',
    '2 зона ВТР и уставка отпуск',
    '3 зона ВТР и уставка отпуск',
    '4 зона ВТР и уставка отпуск',
    '5 зона ВТР и уставка отпуск',
    'шаг балок закалочная печь, сек',
    'шаг балок отпускная печь, сек',
    'Тип предела текучести (1186)'
]

In [7]:
table_for_optimize

,Гр. прочн.,марка стали,диаметр,толщина стенки,Предел текучести нижняя граница,Предел текучести верхняя граница,Предел прочности нижняя граница,Предел прочности верхняя граница,1 зона по ВТР закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,"Скорость прохождения трубы через спрейер, м/с",1 зона ВТР и уставка отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,"шаг балок закалочная печь, сек","шаг балок отпускная печь, сек",Тип предела текучести (1186)
0,К,13ХФА,72.0,5.5,40.0,NaN,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,К,30Г2,118.0,12.0,50.0,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,R95,18ХМФБ,89.0,9.0,66.8,88.0,70.0,90.0,840.0,840.0,840.0,0.2,670.0,670.0,657.0,657.0,637.0,28.0,28.0,0.2
3,R95,Цу12,89.0,9.0,77.3,NaN,70.0,90.0,840.0,840.0,840.0,0.2,670.0,670.0,657.0,657.0,637.0,28.0,28.0,0.2
4,Л,30Г2,114.3,7.0,66.8,87.9,77.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Если верхняя границп не введена, то берем нижнюю + 20

In [8]:
def fill_up(x):
    if pd.isnull(x['Предел текучести верхняя граница']):
        x['Предел текучести верхняя граница'] = x['Предел текучести нижняя граница']+30
    if pd.isnull(x['Предел прочности верхняя граница']):
        x['Предел прочности верхняя граница'] = x['Предел прочности нижняя граница']+30
    return x
table_for_optimize = table_for_optimize.apply(fill_up, axis=1)

In [9]:
table_for_optimize['Прочность середина'] = (table_for_optimize[
    'Предел прочности нижняя граница']+table_for_optimize['Предел прочности верхняя граница'])/2.0
table_for_optimize['Текучесть середина'] = (table_for_optimize[
    'Предел текучести нижняя граница']+table_for_optimize['Предел текучести верхняя граница'])/2.0

In [10]:
table_for_optimize

,Гр. прочн.,марка стали,диаметр,толщина стенки,Предел текучести нижняя граница,Предел текучести верхняя граница,Предел прочности нижняя граница,Предел прочности верхняя граница,1 зона по ВТР закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,"Скорость прохождения трубы через спрейер, м/с",1 зона ВТР и уставка отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,"шаг балок закалочная печь, сек","шаг балок отпускная печь, сек",Тип предела текучести (1186),Прочность середина,Текучесть середина
0,К,13ХФА,72.0,5.5,40.0,70.0,60.0,90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.0,55.00
1,К,30Г2,118.0,12.0,50.0,80.0,70.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,85.0,65.00
2,R95,18ХМФБ,89.0,9.0,66.8,88.0,70.0,90.0,840.0,840.0,840.0,0.2,670.0,670.0,657.0,657.0,637.0,28.0,28.0,0.2,80.0,77.40
3,R95,Цу12,89.0,9.0,77.3,107.3,70.0,90.0,840.0,840.0,840.0,0.2,670.0,670.0,657.0,657.0,637.0,28.0,28.0,0.2,80.0,92.30
4,Л,30Г2,114.3,7.0,66.8,87.9,77.3,107.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,92.3,77.35


### Подтягиваем режим, если он не введен

In [11]:
database = pd.read_csv('prepared_to_saw_gp_del_bath.csv', index_col=0)

In [12]:
database.columns

Index(['№ партии', '№ плавки', 'марка стали', 'диаметр', 'толщина стенки',
       'Гр. прочн.', '1 зона по ВТР закалка', '2 зона по ВТР закалка',
       '3 зона по ВТР закалка', 'шаг балок закалочная печь, сек',
       'Скорость прохождения трубы через спрейер, м/с',
       't˚ C трубы после спреера', '1 зона ВТР и уставка отпуск',
       '2 зона ВТР и уставка отпуск', '3 зона ВТР и уставка отпуск',
       '4 зона ВТР и уставка отпуск', '5 зона ВТР и уставка отпуск',
       'шаг балок отпускная печь, сек', 'C', 'Mn', 'Si', 'P', 'S', 'Cr', 'Ni',
       'Cu', 'Al', 'V', 'Ti', 'Nb', 'Mo', 'N', 'B', 'C-coef',
       'Параметр закалка', 'Параметр отпуск', 'Параметр отпуск новый V',
       'Величина зерна', 'Тип предела текучести (1186)', 'Дата термообработки',
       'ICD', 'Примечание', 'Предел текучести', 'Врем. сопротивление'],
      dtype='object')

In [13]:
database = database[~(database['Скорость прохождения трубы через спрейер, м/с']==1)]
database = database[~(pd.isnull(database['Скорость прохождения трубы через спрейер, м/с']))]

#### Очищаем группу прочности, файл пересохранила, теперь это нужно для самого оптимизатора только

In [14]:
replace_dict_gr = {
    ' ':'',
    '/':'-',
    'ТИП':'тип',
    'К':'K', #русский на английский, везде
    'С':'C',
    'Р':'P',
    'Х':'X',
    'Е':'E',
    'Т':'T',
    'М':'M',
    'У':'Y',
    'Н':'H',
    'В':'B',
    'А':'A',
    'П':'n',
    'О':'O',
    'Т':'T'
}

def fix_h_group(st):
    st = str(st)
    st = st.upper()
    for it in replace_dict_gr:
        st = st.replace(it, replace_dict_gr[it])
    return st

In [15]:
# database.to_csv('prepared_to_saw_gp_del_bath.csv')

In [16]:
database['Гр. прочн.'] = database['Гр. прочн.'].apply(fix_h_group)
database['Примечание'] = database['Гр. прочн.'].apply(lambda x: None)

# database.to_csv('prepared_to_saw_gp_del_bath.csv')
# database

In [17]:
# database[ls_columns_output+targets].to_csv('prepared_to_saw_gp_del_bath.csv')

#### Ищем ближайший режим

In [18]:
def close_value(database, col, value):
    database['diff'] = np.abs(database[col]-value)
    return database.loc[(database['diff']).argmin(),:][col]

In [19]:
def find_row_close_sort(database, row):
    for col in ls_opt_need:
        tmp = database[database[col]==row[col]]
        if tmp.shape[0]>0:
            database = tmp
        else:
            try:
                value = close_value(database, col, row[col])
                database = database[database[col]==value]
            except TypeError:
                database = database[database[col]==row[col].split('-')[0]]
                if database.shape[0]==0:
                    tmp = row
                    tmp[[col for col in row.index if col not in ls_opt_need]] = None
                    tmp['№ партии'] = 'Error!!! (причина:'+ col+')'
                    return tmp               
    return pd.Series(database[database['Дата термообработки'] == database['Дата термообработки'].max()].iloc[0, :])

In [20]:
def find_close_sort(database, df):
    df['Гр. прочн.'] = df['Гр. прочн.'].apply(fix_h_group)
    df = df.apply(lambda x: find_row_close_sort(database, x), axis=1)
    return df

In [52]:
answ = find_close_sort(database, table_for_optimize)

In [53]:
answ

,1 зона ВТР и уставка отпуск,1 зона по ВТР закалка,2 зона ВТР и уставка отпуск,2 зона по ВТР закалка,3 зона ВТР и уставка отпуск,3 зона по ВТР закалка,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,Al,B,C,C-coef,Cr,Cu,ICD,Mn,Mo,N,Nb,Ni,P,S,Si,Ti,V,diff,t˚ C трубы после спреера,Величина зерна,Врем. сопротивление,Гр. прочн.,Дата термообработки,Параметр закалка,Параметр отпуск,Параметр отпуск новый V,Предел прочности верхняя граница,Предел прочности нижняя граница,Предел текучести,Предел текучести верхняя граница,Предел текучести нижняя граница,Примечание,Прочность середина,"Скорость прохождения трубы через спрейер, м/с",Текучесть середина,Тип предела текучести (1186),диаметр,марка стали,толщина стенки,"шаг балок закалочная печь, сек","шаг балок отпускная печь, сек",№ партии,№ плавки
0,525.0,840.0,525.0,840.0,522.0000,850.0,522.0,521.0,0.033,0.00094,0.14,0.348000,0.53,0.10,0.579200,0.48,0.007,0.00800,0.00355,0.05,0.007,0.003,0.24,0.0017,0.053,16.90,37.5,12.986261,62.500000,K,2014-12-13,838.854999,15.844163,11.137841,NaN,NaN,47.000000,NaN,NaN,NaN,NaN,0.180000,NaN,0.2,88.90,13ХФА,12.20,36.000,36.0000,412197,z20302
1,525.0,850.0,580.0,850.0,522.0000,860.0,574.0,521.0,0.030,0.00050,0.29,0.527267,0.06,0.08,1.198637,1.29,0.005,0.00892,0.00553,0.05,0.012,0.008,0.23,0.0040,0.003,44.98,42.5,17.473343,80.000000,K,2013-12-22,847.359565,16.143345,10.032797,NaN,NaN,59.000000,NaN,NaN,NaN,NaN,0.230000,NaN,0.2,73.02,30Г2,10.50,34.000,34.0000,312280,v22228
2,670.0,870.0,670.0,870.0,659.0000,880.0,659.0,642.0,0.031,NaN,0.16,0.523333,1.02,0.19,1.682538,0.56,0.150,0.01000,0.04000,0.14,0.010,0.007,0.23,0.0070,0.070,15.98,70.0,3.706300,86.760204,R95,2018-04-11,861.528411,18.202860,11.362842,NaN,NaN,81.632653,NaN,NaN,NaN,NaN,0.700000,NaN,0.5,73.02,18ХМФБ,5.51,27.000,27.0000,804140,8f1869
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,R95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.00,Цу12,9.00,NaN,NaN,Error!!! (причина:марка стали),NaN
4,567.5,850.0,570.0,850.0,561.1875,860.0,564.0,552.5,0.030,0.00050,0.28,0.551267,0.09,0.19,1.450899,1.38,0.010,0.00800,0.00200,0.12,0.007,0.006,0.25,0.0100,0.003,NaN,42.5,13.518191,82.500000,Л,2016-11-12,847.442750,16.509568,10.064960,NaN,NaN,74.000000,NaN,NaN,NaN,NaN,0.386875,NaN,0.2,114.30,30Г2,7.00,34.125,32.5625,611199,6f3508


#### Оставляем введенные режимы

Немного костыльно, если будет время нкужно будент поменять местами, сначала оставляем те, что уже введены, потом для оставшихся ищем режим.

In [54]:
try_ = table_for_optimize.copy()
try_ = try_.dropna()

In [55]:
try_

,Гр. прочн.,марка стали,диаметр,толщина стенки,Предел текучести нижняя граница,Предел текучести верхняя граница,Предел прочности нижняя граница,Предел прочности верхняя граница,1 зона по ВТР закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,"Скорость прохождения трубы через спрейер, м/с",1 зона ВТР и уставка отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,"шаг балок закалочная печь, сек","шаг балок отпускная печь, сек",Тип предела текучести (1186),Прочность середина,Текучесть середина
2,R95,18ХМФБ,89.0,9.0,66.8,88.0,70.0,90.0,840.0,840.0,840.0,0.2,670.0,670.0,657.0,657.0,637.0,28.0,28.0,0.2,80.0,77.4
3,R95,Цу12,89.0,9.0,77.3,107.3,70.0,90.0,840.0,840.0,840.0,0.2,670.0,670.0,657.0,657.0,637.0,28.0,28.0,0.2,80.0,92.3


In [56]:
def get_index_diff(df1,df2):
    return list(set(df1.index).difference(set(df2.index)))

In [57]:
diff = get_index_diff(answ, try_)

In [58]:
answ = pd.concat([answ.take(diff), try_])
answ = answ[ls_columns_output]

In [59]:
answ

,№ партии,№ плавки,марка стали,диаметр,толщина стенки,Гр. прочн.,1 зона по ВТР закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,"шаг балок закалочная печь, сек","Скорость прохождения трубы через спрейер, м/с",t˚ C трубы после спреера,1 зона ВТР и уставка отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,"шаг балок отпускная печь, сек",C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,B,C-coef,Параметр закалка,Параметр отпуск,Параметр отпуск новый V,Величина зерна,Тип предела текучести (1186),Дата термообработки,ICD,Примечание
0,412197,z20302,13ХФА,88.90,12.2,K,840.0,840.0,850.0,36.000,0.180000,37.5,525.0,525.0,522.0000,522.0,521.0,36.0000,0.14,0.48,0.24,0.007,0.003,0.53,0.05,0.10,0.033,0.053,0.0017,0.00355,0.007,0.00800,0.00094,0.348000,838.854999,15.844163,11.137841,12.986261,0.2,2014-12-13,0.579200,NaN
1,312280,v22228,30Г2,73.02,10.5,K,850.0,850.0,860.0,34.000,0.230000,42.5,525.0,580.0,522.0000,574.0,521.0,34.0000,0.29,1.29,0.23,0.012,0.008,0.06,0.05,0.08,0.030,0.003,0.0040,0.00553,0.005,0.00892,0.00050,0.527267,847.359565,16.143345,10.032797,17.473343,0.2,2013-12-22,1.198637,NaN
4,611199,6f3508,30Г2,114.30,7.0,Л,850.0,850.0,860.0,34.125,0.386875,42.5,567.5,570.0,561.1875,564.0,552.5,32.5625,0.28,1.38,0.25,0.007,0.006,0.09,0.12,0.19,0.030,0.003,0.0100,0.00200,0.010,0.00800,0.00050,0.551267,847.442750,16.509568,10.064960,13.518191,0.2,2016-11-12,1.450899,NaN
2,NaN,NaN,18ХМФБ,89.00,9.0,R95,840.0,840.0,840.0,28.000,0.200000,NaN,670.0,670.0,657.0000,657.0,637.0,28.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2,NaN,NaN,NaN
3,NaN,NaN,Цу12,89.00,9.0,R95,840.0,840.0,840.0,28.000,0.200000,NaN,670.0,670.0,657.0000,657.0,637.0,28.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2,NaN,NaN,NaN


In [60]:
answ = pd.concat([answ,table_for_optimize[['Прочность середина', 'Текучесть середина']]], axis=1)

In [61]:
answ

,№ партии,№ плавки,марка стали,диаметр,толщина стенки,Гр. прочн.,1 зона по ВТР закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,"шаг балок закалочная печь, сек","Скорость прохождения трубы через спрейер, м/с",t˚ C трубы после спреера,1 зона ВТР и уставка отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,"шаг балок отпускная печь, сек",C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,B,C-coef,Параметр закалка,Параметр отпуск,Параметр отпуск новый V,Величина зерна,Тип предела текучести (1186),Дата термообработки,ICD,Примечание,Прочность середина,Текучесть середина
0,412197,z20302,13ХФА,88.90,12.2,K,840.0,840.0,850.0,36.000,0.180000,37.5,525.0,525.0,522.0000,522.0,521.0,36.0000,0.14,0.48,0.24,0.007,0.003,0.53,0.05,0.10,0.033,0.053,0.0017,0.00355,0.007,0.00800,0.00094,0.348000,838.854999,15.844163,11.137841,12.986261,0.2,2014-12-13,0.579200,NaN,75.0,55.00
1,312280,v22228,30Г2,73.02,10.5,K,850.0,850.0,860.0,34.000,0.230000,42.5,525.0,580.0,522.0000,574.0,521.0,34.0000,0.29,1.29,0.23,0.012,0.008,0.06,0.05,0.08,0.030,0.003,0.0040,0.00553,0.005,0.00892,0.00050,0.527267,847.359565,16.143345,10.032797,17.473343,0.2,2013-12-22,1.198637,NaN,85.0,65.00
2,NaN,NaN,18ХМФБ,89.00,9.0,R95,840.0,840.0,840.0,28.000,0.200000,NaN,670.0,670.0,657.0000,657.0,637.0,28.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2,NaN,NaN,NaN,80.0,77.40
3,NaN,NaN,Цу12,89.00,9.0,R95,840.0,840.0,840.0,28.000,0.200000,NaN,670.0,670.0,657.0000,657.0,637.0,28.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2,NaN,NaN,NaN,80.0,92.30
4,611199,6f3508,30Г2,114.30,7.0,Л,850.0,850.0,860.0,34.125,0.386875,42.5,567.5,570.0,561.1875,564.0,552.5,32.5625,0.28,1.38,0.25,0.007,0.006,0.09,0.12,0.19,0.030,0.003,0.0100,0.00200,0.010,0.00800,0.00050,0.551267,847.442750,16.509568,10.064960,13.518191,0.2,2016-11-12,1.450899,NaN,92.3,77.35


###  Подтягиваем среднюю химию 

In [62]:
answ = mean_chem(answ)

Убираем строчки без химии после подтягивания

In [63]:
answ = answ[~answ['C'].isnull()]
answ[ls_chem] = answ[ls_chem].fillna(0)

In [64]:
answ = calc_all_features(answ)
answ = len_pipe(answ)
answ = calc_AC(answ)

In [87]:
answ.reset_index(inplace=True, drop=True)
answ

,№ партии,№ плавки,марка стали,диаметр,толщина стенки,Гр. прочн.,1 зона по ВТР закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,"шаг балок закалочная печь, сек","Скорость прохождения трубы через спрейер, м/с",t˚ C трубы после спреера,1 зона ВТР и уставка отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,"шаг балок отпускная печь, сек",C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,B,C-coef,Параметр закалка,Параметр отпуск,Параметр отпуск новый V,Величина зерна,Тип предела текучести (1186),Дата термообработки,ICD,Примечание,Прочность середина,Текучесть середина,Параметр отпуск новый,Параметр отпуск новый 2,T,sum_piXi,sum_qiXi,длина трубы,W,AC3,AC1
0,412197,z20302,13ХФА,88.90,12.2,K,840.0,840.0,850.0,36.000,0.180000,37.5,525.0,525.0,522.0000,522.0,521.0,36.0000,0.14000,0.48000,0.24000,0.00700,0.00300,0.53000,0.05000,0.1000,0.03300,0.05300,0.00170,0.00355,0.00700,0.00800,0.00094,0.348000,838.854999,15.844163,11.137841,12.986261,0.2,2014-12-13,0.579200,NaN,75.0,55.00,49.920500,11.618726,845.0,-2.927149,-24968.70310,5,0,865.5,733.0
1,312280,v22228,30Г2,73.02,10.5,K,850.0,850.0,860.0,34.000,0.230000,42.5,525.0,580.0,522.0000,574.0,521.0,34.0000,0.29000,1.29000,0.23000,0.01200,0.00800,0.06000,0.05000,0.0800,0.03000,0.00300,0.00400,0.00553,0.00500,0.00892,0.00050,0.527267,847.359565,16.143345,10.032797,17.473343,0.2,2013-12-22,1.198637,NaN,85.0,65.00,49.860488,10.418611,855.0,-3.038328,-27300.77346,5,0,810.0,714.5
2,NaN,NaN,18ХМФБ,89.00,9.0,R95,840.0,840.0,840.0,28.000,0.200000,NaN,670.0,670.0,657.0000,657.0,637.0,28.0000,0.17656,0.57057,0.27012,0.00848,0.00582,0.99551,0.12524,0.1733,0.03268,0.07311,0.01323,0.04006,0.14019,0.00834,0.00000,0.533320,823.567356,18.177488,11.176230,2.502955,0.2,NaN,1.843581,NaN,80.0,77.40,49.666391,11.783254,840.0,1.471933,30276.15789,4,0,858.5,745.0
3,611199,6f3508,30Г2,114.30,7.0,Л,850.0,850.0,860.0,34.125,0.386875,42.5,567.5,570.0,561.1875,564.0,552.5,32.5625,0.28000,1.38000,0.25000,0.00700,0.00600,0.09000,0.12000,0.1900,0.03000,0.00300,0.01000,0.00200,0.01000,0.00800,0.00050,0.551267,847.442750,16.509568,10.064960,13.518191,0.2,2016-11-12,1.450899,NaN,92.3,77.35,49.816823,10.460386,855.0,-4.258518,-36421.41100,10,0,808.5,714.0


In [66]:
# answ = pd.DataFrame(answ.iloc[3,:]).T

### Загрузка моделей для предсказания свойств

In [67]:
def load_model(dir_name, target):
    model = pickle.load(open(dir_name+'/RF_model_'+target+'.sav', 'rb'))
    ls_need_col = json.load(open(dir_name+'/ls_need_col', "r"))
    try:
        scaler = StandardScaler()
        scale_data = json.load(open(dir_name+'/scaler', "r"))
        scaler.mean_ = scale_data[0]
        scaler.scale_ = scale_data[1]
    except:
        scaler=None
    return model, ls_need_col, scaler

In [68]:
models, ls_need_cols, scalers = [], [], []
for target, dir_name in zip(targets, dir_names):
    model, ls_need_col, scaler = load_model(dir_name, target)
    models.append(model)
    ls_need_cols.append(ls_need_col)
    scalers.append(scaler)

### Подготовка шагов для оптимизатора

Возникла проблема, оптимизатор не работает с двумя моделями, потому что они обучены на разном подмножестве признаков, сейчас пробую обойти это, если не получится, то нужно будет переобучить модельки.

Придумала костыль, живем дальше

In [88]:
ls_temp_z = [
    '1 зона по ВТР закалка', '2 зона по ВТР закалка',
    '3 зона по ВТР закалка'
]

ls_temp_o = [
    '1 зона ВТР и уставка отпуск',
    '2 зона ВТР и уставка отпуск', '3 зона ВТР и уставка отпуск',
    '4 зона ВТР и уставка отпуск', '5 зона ВТР и уставка отпуск'
]

# ls_s_spr = ['Скорость прохождения трубы через спрейер, м/с']

ls_shag = [
    'шаг балок закалочная печь, сек', 'шаг балок отпускная печь, сек'
]

ls_fit_param = ls_temp_o+ls_temp_z+ls_shag

In [131]:
answ.columns

Index(['№ партии', '№ плавки', 'марка стали', 'диаметр', 'толщина стенки',
       'Гр. прочн.', '1 зона по ВТР закалка', '2 зона по ВТР закалка',
       '3 зона по ВТР закалка', 'шаг балок закалочная печь, сек',
       'Скорость прохождения трубы через спрейер, м/с',
       't˚ C трубы после спреера', '1 зона ВТР и уставка отпуск',
       '2 зона ВТР и уставка отпуск', '3 зона ВТР и уставка отпуск',
       '4 зона ВТР и уставка отпуск', '5 зона ВТР и уставка отпуск',
       'шаг балок отпускная печь, сек', 'C', 'Mn', 'Si', 'P', 'S', 'Cr', 'Ni',
       'Cu', 'Al', 'V', 'Ti', 'Nb', 'Mo', 'N', 'B', 'C-coef',
       'Параметр закалка', 'Параметр отпуск', 'Параметр отпуск новый V',
       'Величина зерна', 'Тип предела текучести (1186)', 'Дата термообработки',
       'ICD', 'Примечание', 'Прочность середина', 'Текучесть середина',
       'Параметр отпуск новый', 'Параметр отпуск новый 2', 'T', 'sum_piXi',
       'sum_qiXi', 'длина трубы', 'W', 'AC3', 'AC1'],
      dtype='object')

In [134]:
answ[ls_need_union + ['AC3', 'AC1']]

,1 зона по ВТР закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,"Скорость прохождения трубы через спрейер, м/с",t˚ C трубы после спреера,1 зона ВТР и уставка отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,"шаг балок закалочная печь, сек","шаг балок отпускная печь, сек",диаметр,толщина стенки,C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,Параметр закалка,Параметр отпуск новый V,C-coef,Величина зерна,Параметр отпуск,AC3,AC1
0,840.0,840.0,850.0,0.180000,37.5,525.0,525.0,522.0000,522.0,521.0,36.000,36.0000,88.90,12.2,0.14000,0.48000,0.24000,0.00700,0.00300,0.53000,0.05000,0.1000,0.03300,0.05300,0.00170,0.00355,0.00700,0.00800,838.854999,11.137841,0.348000,12.986261,15.844163,865.5,733.0
1,850.0,850.0,860.0,0.230000,42.5,525.0,580.0,522.0000,574.0,521.0,34.000,34.0000,73.02,10.5,0.29000,1.29000,0.23000,0.01200,0.00800,0.06000,0.05000,0.0800,0.03000,0.00300,0.00400,0.00553,0.00500,0.00892,847.359565,10.032797,0.527267,17.473343,16.143345,810.0,714.5
2,840.0,840.0,840.0,0.200000,NaN,670.0,670.0,657.0000,657.0,637.0,28.000,28.0000,89.00,9.0,0.17656,0.57057,0.27012,0.00848,0.00582,0.99551,0.12524,0.1733,0.03268,0.07311,0.01323,0.04006,0.14019,0.00834,823.567356,11.176230,0.533320,2.502955,18.177488,858.5,745.0
3,850.0,850.0,860.0,0.386875,42.5,567.5,570.0,561.1875,564.0,552.5,34.125,32.5625,114.30,7.0,0.28000,1.38000,0.25000,0.00700,0.00600,0.09000,0.12000,0.1900,0.03000,0.00300,0.01000,0.00200,0.01000,0.00800,847.442750,10.064960,0.551267,13.518191,16.509568,808.5,714.0


In [136]:
X_c_down = calc_all_features(answ[ls_need_union + ['AC3', 'AC1']]).dropna().copy()
for col in ls_temp_z:
    X_c_down[col] = X_c_down['AC3']

X_c_down[ls_temp_o] = 500
X_c_down[ls_shag] = 24

# X_c_down['Скорость прохождения трубы через спрейер, м/с'] = X_c_down[
#     'Скорость прохождения трубы через спрейер, м/с'].apply(lambda x: x if x>0 else 0)

X_c_up = calc_all_features(answ[ls_need_union + ['AC3', 'AC1']]).dropna().copy()
X_c_up[ls_temp_z] = 950
for col in ls_temp_o:
    X_c_up[col] = X_c_up['AC1']

In [137]:
def uniq(alist):    # Fastest order preserving
    set = {}
    return [set.setdefault(e,e) for e in alist if e not in set]

ls_need_union = uniq(list(sum(ls_need_cols, [])))

In [138]:
# X_c_up = X_c_up[ls_need_union].dropna()
# X_c_down = X_c_down[ls_need_union].dropna()

Добавлю еще один костыль в виде самостоятельной првоерки по границам.....


In [139]:
def is_in_bounds(row, bounds):
    penalty = 0
    for i in range(len(bounds)):
        if bounds[i][0]<=row.iloc[:, i].values<=bounds[i][1]:
            pass
        else:
            penalty += 100
    return penalty

In [140]:
# Это нужно?
# df = pd.DataFrame(df, index=ls_need_union).T

In [141]:
# tmp_try_headers = answ_n[ls_fit_param].iloc[0,:].values
# pd.DataFrame(tmp_try_headers, index=ls_fit_param).T
# tmp_try_headers.shape
# ls_fit_param

In [228]:
def model_pr(fit_params, all_params, bounds, eta):
    centr_ys = all_params['Текучесть середина']
    centr_h = all_params['Прочность середина']
    all_params = pd.concat([all_params[list(set(ls_need_union + ['длина трубы'])-set(ls_fit_param))],
                            pd.Series(fit_params, index=ls_fit_param)])
    fit_params = pd.DataFrame(fit_params, index=ls_fit_param).T
    score = 0
    
# пересчет параметров
    if pd.DataFrame(all_params).shape[0] > pd.DataFrame(all_params).shape[1]:
        all_params = pd.DataFrame(all_params).T
    else:
        all_params = pd.DataFrame(all_params)
    all_params = calc_all_features(all_params)
    all_params = new_spr(all_params)
# попытки победить ValueError: Input contains NaN, infinity or a value too large for dtype('float32').
    all_params.reset_index(inplace=True, drop=True)
    all_params.dropna(inplace=True)
    all_params = all_params.astype(np.float32)
    for model_name, model, ls_need_col, scaler in zip(['ys','h'], models, ls_need_cols, scalers):
        if model_name=='h':
            centr=centr_h
            pred = model.predict(all_params[ls_need_col])
            tmp_score = np.abs(pred - centr)
        else:
            centr=centr_ys
            pred = model.predict(all_params[ls_need_col])
            tmp_score = np.abs(pred - centr)
        
        if tmp_score<2:
            tmp_score=0
        if tmp_score > 5:
#             tmp_score+=float('inf')
            tmp_score += 100
        
        score += tmp_score
        
    score += max(np.abs(all_params['шаг балок закалочная печь, сек'].values-24),
                 np.abs(all_params['шаг балок отпускная печь, сек'].values-24))*eta
    score += is_in_bounds(fit_params, bounds)
    return score

In [190]:
answ_n = answ[ls_need_union + models_bonds + ['длина трубы']].dropna()
# answ_n = answ.dropna().copy()

In [230]:
from scipy.optimize import brute

In [238]:
%%time
answers_array = []
X_a = calc_all_features(X_c_down[ls_need_union]).copy()
X_b = calc_all_features(X_c_up[ls_need_union]).copy()
answ_n.reset_index(inplace=True, drop=True)
X_a.reset_index(inplace=True, drop=True)
X_b.reset_index(inplace=True, drop=True)
answ_n.reset_index(inplace=True, drop=True)
for it in X_a.index:
    bounds = [(i, j) for i, j in zip(X_a.loc[it,ls_fit_param], X_b.loc[it,ls_fit_param])]
    print(answ_n.iloc[it,-3],answ_n.iloc[it,-2])
    all_params = answ_n.iloc[it,:]
    fit_p = answ_n.loc[it, ls_fit_param]
    
#     a = minimize(lambda fit_params: model_pr(fit_params, 
#                                              all_params,
#                                              bounds, 
#                                              10),
#                  fit_p, method='TNC', bounds=bounds, tol=1e-10)


    a = fmin(lambda fit_params: model_pr(fit_params, 
                                             all_params,
                                             bounds, 
                                             10),
                 fit_p)

#     a = fmin(lambda fit_params: model_pr(fit_params, 
#                                              all_params,
#                                              bounds, 
#                                              10),
#                  fit_p)

#     a = differential_evolution(lambda fit_params: model_pr(fit_params,all_params,bounds,10), bounds)


#     a = brute(lambda fit_params: model_pr(fit_params,all_params,bounds,10), bounds, finish=optimize.fmin)
    
    print('ВАПРОС a', it, models[0].predict(X_a.loc[it,ls_need_cols[0]].values.reshape(1, -1)),
         models[1].predict(X_a.loc[it,ls_need_cols[1]].values.reshape(1, -1)))
    print('ВАПРОС origin', it, models[0].predict(answ_n.loc[it,ls_need_cols[0]].values.reshape(1, -1)),
         models[1].predict(answ_n.loc[it,ls_need_cols[1]].values.reshape(1, -1)))
    print('ВАПРОС b', it, models[0].predict(X_b.loc[it,ls_need_cols[0]].values.reshape(1, -1)),
         models[1].predict(X_b.loc[it,ls_need_cols[1]].values.reshape(1, -1)))
    print(a)
    all_params = pd.concat([all_params[set(ls_need_union+['длина трубы'])-set(ls_fit_param)],pd.Series(a, index=ls_fit_param)])
    answers_array.append(all_params)
#     tmp_a_x = a['x']
#     df_h = np.concatenate((tmp_a_x[:2],tmp_a_x[3:-2],tmp_a_x[-1:]))
#     df_ys = tmp_a_x[:-1]
    ys = models[0].predict(calc_all_features(pd.DataFrame(answers_array[-1]).T)[ls_need_cols[0]])
    h = models[1].predict(calc_all_features(pd.DataFrame(answers_array[-1]).T)[ls_need_cols[1]])
    answers_array[-1] = pd.concat([answers_array[-1], pd.Series([ys[0],h[0]], index=['pred Текучесть', 'pred Прочность'])])
    print('АТВЕТ ', it, ys, h)

55.0 75.0
Optimization terminated successfully.
         Current function value: 145.960979
         Iterations: 676
         Function evaluations: 1160
ВАПРОС a 0 [54.76426506] [67.45032857]
ВАПРОС origin 0 [49.87091482] [67.26786259]
ВАПРОС b 0 [41.23256195] [58.27635147]
[645.26543786 517.67791098 533.57958197 500.0000287  640.32338689
 928.00182079 924.59351605 912.82542448  24.00001259  27.20549913]
АТВЕТ  0 [51.26382153] [65.57804422]
65.0 85.0
ВАПРОС a 1 [79.6162958] [93.57698479]
ВАПРОС origin 1 [69.34388645] [84.04754751]
ВАПРОС b 1 [52.54503792] [72.34309223]
[608.07445288 592.57988896 554.11110885 598.76024117 515.25313855
 830.74506947 847.50021563 870.63874525  29.42887538  28.64786094]
АТВЕТ  1 [69.91599071] [84.21312453]
77.35 92.3
Optimization terminated successfully.
         Current function value: 13.586057
         Iterations: 426
         Function evaluations: 833
ВАПРОС a 2 [79.77398813] [92.96049525]
ВАПРОС origin 2 [74.90503835] [83.710186]
ВАПРОС b 2 [55.700990

In [193]:
opt_answer_end = pd.DataFrame(answers_array)
opt_answer_end = calc_all_features(opt_answer_end)
opt_answer_end = new_spr(opt_answer_end)
opt_answer_end = opt_answer_end[ls_need_cols[0]+['pred Текучесть','pred Прочность']].copy()

In [194]:
X_a

,1 зона по ВТР закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,"Скорость прохождения трубы через спрейер, м/с",t˚ C трубы после спреера,1 зона ВТР и уставка отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,"шаг балок закалочная печь, сек","шаг балок отпускная печь, сек",диаметр,толщина стенки,C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,Параметр закалка,Параметр отпуск новый V,C-coef,Величина зерна,Параметр отпуск,ICD,Параметр отпуск новый,Параметр отпуск новый 2,T,sum_piXi,sum_qiXi
0,865.5,865.5,865.5,0.180000,37.5,500.0,500.0,500.0,500.0,500.0,24.0,24.0,88.90,12.2,0.14,0.48,0.24,0.007,0.003,0.53,0.05,0.10,0.033,0.053,0.0017,0.00355,0.007,0.00800,844.822498,10.520742,0.348000,13.791337,15.098746,0.579200,49.514522,10.988516,865.5,-2.927149,-24968.70310
1,810.0,810.0,810.0,0.230000,42.5,500.0,500.0,500.0,500.0,500.0,24.0,24.0,73.02,10.5,0.29,1.29,0.23,0.012,0.008,0.06,0.05,0.08,0.030,0.003,0.0040,0.00553,0.005,0.00892,791.272768,9.281685,0.527267,13.394875,15.098746,1.198637,49.510877,9.648969,810.0,-3.038328,-27300.77346
2,808.5,808.5,808.5,0.386875,42.5,500.0,500.0,500.0,500.0,500.0,24.0,24.0,114.30,7.0,0.28,1.38,0.25,0.007,0.006,0.09,0.12,0.19,0.030,0.003,0.0100,0.00200,0.010,0.00800,789.824161,9.112796,0.551267,10.717220,15.098746,1.450899,49.510038,9.480080,808.5,-4.258518,-36421.41100


In [195]:
answ_n

,1 зона по ВТР закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,"Скорость прохождения трубы через спрейер, м/с",t˚ C трубы после спреера,1 зона ВТР и уставка отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,"шаг балок закалочная печь, сек","шаг балок отпускная печь, сек",диаметр,толщина стенки,C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,Параметр закалка,Параметр отпуск новый V,C-coef,Величина зерна,Параметр отпуск,Текучесть середина,Прочность середина,длина трубы
0,840.0,840.0,850.0,0.180000,37.5,525.0,525.0,522.0000,522.0,521.0,36.000,36.0000,88.90,12.2,0.14,0.48,0.24,0.007,0.003,0.53,0.05,0.10,0.033,0.053,0.0017,0.00355,0.007,0.00800,838.854999,11.137841,0.348000,12.986261,15.844163,55.00,75.0,5
1,850.0,850.0,860.0,0.230000,42.5,525.0,580.0,522.0000,574.0,521.0,34.000,34.0000,73.02,10.5,0.29,1.29,0.23,0.012,0.008,0.06,0.05,0.08,0.030,0.003,0.0040,0.00553,0.005,0.00892,847.359565,10.032797,0.527267,17.473343,16.143345,65.00,85.0,5
2,850.0,850.0,860.0,0.386875,42.5,567.5,570.0,561.1875,564.0,552.5,34.125,32.5625,114.30,7.0,0.28,1.38,0.25,0.007,0.006,0.09,0.12,0.19,0.030,0.003,0.0100,0.00200,0.010,0.00800,847.442750,10.064960,0.551267,13.518191,16.509568,77.35,92.3,10


In [196]:
opt_answer_end

,1 зона по ВТР закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,"Скорость прохождения трубы через спрейер, м/с",t˚ C трубы после спреера,1 зона ВТР и уставка отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,"шаг балок закалочная печь, сек","шаг балок отпускная печь, сек",диаметр,толщина стенки,C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,Параметр закалка,Параметр отпуск новый V,C-coef,Величина зерна,pred Прочность,pred Текучесть
0,913.333671,948.740198,949.999992,0.312500,37.5,725.264690,622.417474,505.293954,503.434652,606.772001,24.000001,27.443273,88.90,12.2,0.14,0.48,0.24,0.007,0.003,0.53,0.05,0.10,0.033,0.053,0.0017,0.00355,0.007,0.00800,926.171319,11.153536,0.348000,20.316886,65.220102,50.747355
1,841.060584,842.225443,909.269218,0.243902,42.5,521.145641,641.210017,566.747835,617.105315,500.180552,28.500002,24.025357,73.02,10.5,0.29,1.29,0.23,0.012,0.008,0.06,0.05,0.08,0.030,0.003,0.0040,0.00553,0.005,0.00892,891.177544,10.019152,0.527267,18.937267,82.124097,69.267591
2,945.526007,890.652412,838.434759,0.196518,42.5,643.258566,578.813417,587.944458,529.509519,547.190707,29.442912,29.442910,114.30,7.0,0.28,1.38,0.25,0.007,0.006,0.09,0.12,0.19,0.030,0.003,0.0100,0.00200,0.010,0.00800,823.137111,9.929006,0.551267,13.821179,90.458260,76.410062


In [197]:
X_b

,1 зона по ВТР закалка,2 зона по ВТР закалка,3 зона по ВТР закалка,"Скорость прохождения трубы через спрейер, м/с",t˚ C трубы после спреера,1 зона ВТР и уставка отпуск,2 зона ВТР и уставка отпуск,3 зона ВТР и уставка отпуск,4 зона ВТР и уставка отпуск,5 зона ВТР и уставка отпуск,"шаг балок закалочная печь, сек","шаг балок отпускная печь, сек",диаметр,толщина стенки,C,Mn,Si,P,S,Cr,Ni,Cu,Al,V,Ti,Nb,Mo,N,Параметр закалка,Параметр отпуск новый V,C-coef,Величина зерна,Параметр отпуск,ICD,Параметр отпуск новый,Параметр отпуск новый 2,T,sum_piXi,sum_qiXi
0,950.0,950.0,950.0,0.180000,37.5,733.0,733.0,733.0,733.0,733.0,36.000,36.0000,88.90,12.2,0.14,0.48,0.24,0.007,0.003,0.53,0.05,0.10,0.033,0.053,0.0017,0.00355,0.007,0.00800,936.793435,14.099834,0.348000,21.275278,20.057753,0.579200,49.924188,14.708605,950.0,-2.927149,-24968.70310
1,950.0,950.0,950.0,0.230000,42.5,714.5,714.5,714.5,714.5,714.5,34.000,34.0000,73.02,10.5,0.29,1.29,0.23,0.012,0.008,0.06,0.05,0.08,0.030,0.003,0.0040,0.00553,0.005,0.00892,935.284657,12.201215,0.527267,26.786855,19.632454,1.198637,49.863915,12.670417,950.0,-3.038328,-27300.77346
2,950.0,950.0,950.0,0.386875,42.5,714.0,714.0,714.0,714.0,714.0,34.125,32.5625,114.30,7.0,0.28,1.38,0.25,0.007,0.006,0.09,0.12,0.19,0.030,0.003,0.0100,0.00200,0.010,0.00800,935.381411,11.936755,0.551267,19.210719,19.579876,1.450899,49.820050,12.405719,950.0,-4.258518,-36421.41100
